# CNN Fruits

In [47]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
from tensorflow.keras.optimizers import RMSprop
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split

In [48]:
img_width, img_height = 224, 224 
train_data_dir = '../input/fruits/fruits-360/Training'
validation_data_dir = '../input/fruits/fruits-360/Test'
batch_size = 16

In [49]:
fruit_list = ["Kiwi", "Banana", "Orange",
                "Limes", "Lemon","Pear", "Pear 2", "Papaya","Apple Golden 1","Apple Golden 2",
              "Apple Golden 3","Apple Braeburn","Apple Red 1","Apple Red 2","Apple Red 3",
              "Apple Red Yellow 1","Apple Red Yellow 2"
              "Banana","Banana Red","Banana Lady Finger","Corn","Corn Husk","Mango","Mango Red",
              "Strawberry","Strawberry Wedge", "Pineapple", "Pomegranate",
              "Tomato 1","Tomato 2","Tomato 3","Tomato 4","Tomato Heart","Tomato not Ripened",
              "Watermelon"
             ]
print(len(fruit_list))

34


In [50]:
image_generator = ImageDataGenerator(rescale=1/255, validation_split=0.3)    

train_generator = image_generator.flow_from_directory(train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    classes=fruit_list,
    class_mode='categorical')

validation_generator = image_generator.flow_from_directory(validation_data_dir,
    target_size=(img_height, img_width),
    classes=fruit_list,
    batch_size=batch_size,
    class_mode='categorical')

Found 17187 images belonging to 34 classes.
Found 5747 images belonging to 34 classes.


In [51]:
train_generator.class_indices

{'Kiwi': 0,
 'Banana': 1,
 'Orange': 2,
 'Limes': 3,
 'Lemon': 4,
 'Pear': 5,
 'Pear 2': 6,
 'Papaya': 7,
 'Apple Golden 1': 8,
 'Apple Golden 2': 9,
 'Apple Golden 3': 10,
 'Apple Braeburn': 11,
 'Apple Red 1': 12,
 'Apple Red 2': 13,
 'Apple Red 3': 14,
 'Apple Red Yellow 1': 15,
 'Apple Red Yellow 2Banana': 16,
 'Banana Red': 17,
 'Banana Lady Finger': 18,
 'Corn': 19,
 'Corn Husk': 20,
 'Mango': 21,
 'Mango Red': 22,
 'Strawberry': 23,
 'Strawberry Wedge': 24,
 'Pineapple': 25,
 'Pomegranate': 26,
 'Tomato 1': 27,
 'Tomato 2': 28,
 'Tomato 3': 29,
 'Tomato 4': 30,
 'Tomato Heart': 31,
 'Tomato not Ripened': 32,
 'Watermelon': 33}

In [52]:
model = tf.keras.models.Sequential([  ## initializing and making an empty model with sequential
  
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution layer
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(224, 224,3)), ## image input shape is 300x300x3 
                           


    tf.keras.layers.MaxPooling2D(2,2),    # doing max_pooling
    tf.keras.layers.Dropout(0.2),         #16 neurons in this layer

  
    # The second convolution layer
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'), # another layer with 32 neurons
    tf.keras.layers.MaxPooling2D(2,2),     # doing max_pooling
    tf.keras.layers.Dropout(0.2),


    # The third convolution layer
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), # another layer with 64 neurons
    tf.keras.layers.MaxPooling2D(2,2),        # doing max_pooling
    tf.keras.layers.Dropout(0.2),



    # The fourth convolution layer
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), # another layer with 64 neurons
    tf.keras.layers.MaxPooling2D(2,2),          # doing max_pooling
    tf.keras.layers.Dropout(0.2),  


    # The fifth convolution 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), # another layer with 64 neurons
    tf.keras.layers.MaxPooling2D(2,2),        # doing max_pooling
    tf.keras.layers.Dropout(0.2),



    tf.keras.layers.Flatten(),  # reducing layers arrays 
    tf.keras.layers.Dense(512, activation='relu'), # 512 neuron hidden layer



    # Only 1 output neuron. It will contain a value from 0-1 where 0 for ('normal') clas and 
    # 1 for ('pneumonia') class
    tf.keras.layers.Dense(34, activation='softmax')

])

# to get the summary of the model
model.summary()  # summarising a model

# configure the model for traning by adding metrics

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 222, 222, 16)      448       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 111, 111, 16)      0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 109, 109, 32)      4640      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 52, 52, 64)       

In [53]:
# from keras.callbacks import ModelCheckpoint
# filepath="weights.best.hdf5" # mentioning a file for saving checkpoint model in case it gets interrupted

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
# ## we marked filepath, monitor and mentioned to save best model only 


# callbacks_list = [checkpoint]  # customising model to save checkpoints
class MyThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None): 
        val_acc = logs["val_accuracy"]
        if val_acc >= self.threshold:
            self.model.stop_training = True
            print("\nReq acc is reached")
my_callback = MyThresholdCallback(threshold=0.96)

In [54]:

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.001),
              metrics = ['accuracy']) 
print('Compiled !')    # compiling mode

Compiled !


In [55]:
hist = model.fit_generator(
    generator = train_generator,
    steps_per_epoch = 21457//(100),
    epochs = 30,
    shuffle=True,
    validation_data = validation_generator,
    callbacks=[my_callback],
    validation_steps = 7777 // 100
                   )

Epoch 1/30
214/214 [==============================] - 10s 46ms/step - loss: 1.7575 - accuracy: 0.4717 - val_loss: 1.1072 - val_accuracy: 0.6826
Epoch 2/30
214/214 [==============================] - 10s 46ms/step - loss: 0.4952 - accuracy: 0.8470 - val_loss: 0.8082 - val_accuracy: 0.7378
Epoch 3/30
214/214 [==============================] - 9s 44ms/step - loss: 0.2501 - accuracy: 0.9197 - val_loss: 0.7599 - val_accuracy: 0.7670
Epoch 4/30
214/214 [==============================] - 9s 43ms/step - loss: 0.1776 - accuracy: 0.9477 - val_loss: 1.2536 - val_accuracy: 0.5925
Epoch 5/30
214/214 [==============================] - 10s 45ms/step - loss: 0.1559 - accuracy: 0.9603 - val_loss: 0.8142 - val_accuracy: 0.7435
Epoch 6/30
214/214 [==============================] - 10s 49ms/step - loss: 0.0933 - accuracy: 0.9778 - val_loss: 0.5944 - val_accuracy: 0.8571
Epoch 7/30
214/214 [==============================] - 10s 46ms/step - loss: 0.1109 - accuracy: 0.9749 - val_loss: 0.4247 - val_accuracy: 0